In [14]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
import redis

In [2]:
faceapp = FaceAnalysis(name='buffalo_sc', root='', providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0, det_size=(640, 640), det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [15]:
hostname = "127.0.0.1"
port = 6379
r = redis.StrictRedis(hostname, port)

#### Collect Person and Role

In [6]:
person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
  role = input("""
  Please choose
              1. Student
              2. Teacher

              Enter number either 1 or 2
  """)
  if role in ('1', '2'):
    if role == '1':
      role = 'Student'
    else:
      role = 'Teacher'
    break
  else:
    print('Invalid entry, try again')
    if i == 3:
      print('Exceeds maximum trials')

key = person_name + '@' + role

Invalid entry, try again
Invalid entry, try again
Invalid entry, try again


#### Collect Samples

In [10]:
cap = cv2.VideoCapture(0)
face_embeddings = []
sample = 0
while True:
  ret, frame = cap.read()
  if ret == False:
    print('Unable to read camera')
    break

  # get reuslts from insightface model
  results = faceapp.get(frame, max_num=1)
  for res in results:
    sample += 1
    x1, y1, x2, y2 = res['bbox'].astype(int)
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

    # facial features
    embeddings = res['embedding']
    face_embeddings.append(embeddings)
  
  if sample >= 200:
    break

  cv2.imshow('frame', frame)
  if cv2.waitKey(1) == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()

#### Optimize Data & Save in Redis Database

In [11]:
x_mean = np.asarray(face_embeddings).mean(axis=0)

In [12]:
x_mean_bytes = x_mean.tobytes()

In [ ]:
# save key and facial bytes into redis
r.hset(name='academy:register', key=key, value=x_mean_bytes)